In [2]:
from itertools import combinations
from z3 import *
import math

In [3]:
def overlapping(s, size_rec_i, relation, p, i, j):
   
    if size_rec_i >= len(p[j]): 
        s.add(Not(relation[i][j]))
    else:
        s.add(Or(Not(relation[i][j]), Not(p[j][size_rec_i-1])))       
        for elx in range(len(p[i])):
            if elx + size_rec_i + 1 >= len(p[j]):
                s.add(Or(Not(relation[i][j]), p[i][elx]))
                break
            else:
                s.add(Or(Not(relation[i][j]), p[i][elx], Not(p[j][elx + size_rec_i])))

def two_d_spp(r_sizes, plate_width, lower_bound, upper_bound, n_of_rectangles):
    
    height = int((upper_bound + lower_bound)/2)
    print(height)
    lr = [[Bool(f"lr_{i}_{j}") for j in range(1, n_of_rectangles+1)] for i in range(1, n_of_rectangles+1)]
    ud = [[Bool(f"ud_{i}_{j}") for j in range(1, n_of_rectangles+1)] for i in range(1, n_of_rectangles+1)]
    px = [[Bool(f"px_{i+1}_{e}") for e in range(int((plate_width - r_sizes[i][0])/2) + 1 )] 
          if r_sizes[i][1] == max([coord[1] for coord in r_sizes ])
          else [Bool(f"px_{i+1}_{e}") for e in range(plate_width - r_sizes[i][0] + 1 )]
          for i in range(n_of_rectangles)]    
    py = [[Bool(f"py_{i+1}_{f}") for f in range(int((height - r_sizes[i][1])/2) + 1)] 
          if r_sizes[i][1] == max([coord[1] for coord in r_sizes ])
          else [Bool(f"py_{i+1}_{e}") for e in range(height - r_sizes[i][1] + 1 )]
          for i in range(n_of_rectangles)]   
    s = Solver()

    for i in range(n_of_rectangles):
        for elx in range(len(px[i])-1):
            s.add(Or(Not(px[i][elx]), px[i][elx+1]))
        for ely in range(len(py[i])-1): 
            s.add(Or(Not(py[i][ely]), py[i][ely+1]))
            
    for i in range(n_of_rectangles):
        for j in range(i+1, n_of_rectangles):
            s.add(Or(lr[i][j], lr[j][i], ud[i][j], ud[j][i]))

    for i in range(n_of_rectangles):
        for j in range(i+1, n_of_rectangles):
            w_i= r_sizes[i][0]
            w_j= r_sizes[j][0]
            h_i= r_sizes[i][1]
            h_j= r_sizes[j][1]
            
            overlapping(s, w_i,  lr, px, i, j)
            overlapping(s, w_j,  lr, px, j, i)
            overlapping(s, h_i,  ud, py, i, j)
            overlapping(s, h_j,  ud, py, j, i)
           
                    
     
    #reducing the domain of the highest rectangle
    for i in range(n_of_rectangles):
         if r_sizes[i][1] == max([coord[1] for coord in r_sizes ]):
                for j in range(n_of_rectangles):
                    #ho anche j=i qua nota bene
                    if r_sizes[j][0] > int((plate_width - r_sizes[i][0])/2):
                        s.add(Not(lr[j][i]))
                break
    #reducing the domain of the highest rectangle
    for i in range(n_of_rectangles):
         if r_sizes[i][1] == max([coord[1] for coord in r_sizes ]):
                for j in range(n_of_rectangles):
                    if r_sizes[j][1] > int((height - r_sizes[i][1])/2):
                        s.add(Not(ud[j][i]))
                break
    
    #same rectangles (da controllare)
    same_rectangles = []
    for i in range(n_of_rectangles):
        for j in range(i+1,n_of_rectangles):
            if r_sizes[i][0]==r_sizes[j][0] and r_sizes[i][1]==r_sizes[j][1]:
                same_rectangles.append((i,j))
    for rec in same_rectangles:
        i, j = rec 
        s.add(Not(lr[j][i]))
        s.add(Or(lr[i][j], Not(ud[j][i])))
    print(same_rectangles)    
    
    if s.check() == sat and height == lower_bound:
        m = s.model()
        return (height,  [item for sublist in px for item in sublist if m.evaluate(item)==True], 
                         [item for sublist in py for item in sublist if m.evaluate(item)==True],
                         [item for sublist in lr for item in sublist if m.evaluate(item)==True],
                         [item for sublist in ud for item in sublist if m.evaluate(item)==True])
    
                
    
    if s.check()==sat:
        return two_d_spp(r_sizes, plate_width, lower_bound, height, n_of_rectangles)
    return two_d_spp(r_sizes, plate_width, height+1, upper_bound, n_of_rectangles)    
             
#W = 9;
#n_of_rectangles = 5
#sizes = [[3,3],[3,4],[3,5],[3,6],[3,9]]
W = 20
n_of_rectangles = 14
sizes = [[3,3], [3,4], [3,5], [3,6], [3,7], [3,8], [3,9],[3,10],[3,11],[3,17],[4,3],[4,9],[4,11],[4,17]]
lower_bound = max([sizes[i][1] for i in range(n_of_rectangles)])
upper_bound = sum([sizes[i][1] for i in range(n_of_rectangles)])
two_d_spp(sizes, W, lower_bound, upper_bound, n_of_rectangles)

68
[]
42
[]
29
[]
23
[]
20
[]
18
[]
19
[]
20
[]


(20,
 [px_1_0,
  px_1_1,
  px_1_2,
  px_1_3,
  px_1_4,
  px_1_5,
  px_1_6,
  px_1_7,
  px_1_8,
  px_1_9,
  px_1_10,
  px_1_11,
  px_1_12,
  px_1_13,
  px_1_14,
  px_1_15,
  px_1_16,
  px_1_17,
  px_2_7,
  px_2_8,
  px_2_9,
  px_2_10,
  px_2_11,
  px_2_12,
  px_2_13,
  px_2_14,
  px_2_15,
  px_2_16,
  px_2_17,
  px_3_14,
  px_3_15,
  px_3_16,
  px_3_17,
  px_4_7,
  px_4_8,
  px_4_9,
  px_4_10,
  px_4_11,
  px_4_12,
  px_4_13,
  px_4_14,
  px_4_15,
  px_4_16,
  px_4_17,
  px_5_14,
  px_5_15,
  px_5_16,
  px_5_17,
  px_6_14,
  px_6_15,
  px_6_16,
  px_6_17,
  px_8_7,
  px_8_8,
  px_8_9,
  px_8_10,
  px_8_11,
  px_8_12,
  px_8_13,
  px_8_14,
  px_8_15,
  px_8_16,
  px_8_17,
  px_10_4,
  px_10_5,
  px_10_6,
  px_10_7,
  px_10_8,
  px_11_3,
  px_11_4,
  px_11_5,
  px_11_6,
  px_11_7,
  px_11_8,
  px_11_9,
  px_11_10,
  px_11_11,
  px_11_12,
  px_11_13,
  px_11_14,
  px_11_15,
  px_11_16,
  px_12_10,
  px_12_11,
  px_12_12,
  px_12_13,
  px_12_14,
  px_12_15,
  px_12_16,
  px_13_10,
  px_13_1